In [2]:
from __future__ import annotations

import collections
import datetime as dt
import itertools
import random
import pickle

import numpy as np
import pandas as pd

from symbolic_transfer_entropy import symbolic_transfer_entropy

In [3]:
import pickle

In [4]:
import plotly.express as px 
import plotly

plotly.io.templates.defualt='plotly_white'

import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np

In [5]:
import textwrap

In [19]:
with open('/data_users1/crazybrokeasian/projects/trans-fer-entropy/topic_labels_20_finalized.pkl','rb') as f:
    topic_labels_dict_raw=pickle.load(f)

topic_labels_dict={k:v.replace('\n','<br>'
)for k,v in topic_labels_dict_raw.items()}

topic_labels_dict_raw={k:v.replace(' \n',''
)for k,v in topic_labels_dict_raw.items()}
topic_labels_dict_raw

topic_labels_dict_50 = {i: textwrap.fill(topic_labels_dict_raw[i], 50).replace("\n", "<br>") for i in topic_labels_dict_raw}
topic_labels_dict_50


topic_labels_dict_str={str(k):v for k,v in topic_labels_dict.items()}

In [20]:
pd.read_csv('/home/pranavgoel/trans-fer-entropy/topic_modeling/prior_data_apr_jun_2023/all_texts_combined.csv').query('media_name == "Fox News"')['publish_date'].min()

'2023-05-09 16:05:13'

In [103]:
plots_path='/data_users1/crazybrokeasian/projects/trans-fer-entropy/plots_and_images/topic_modeling'

In [21]:
collection_color_map={'nyt': '#2E498A', 'newyork': '#4B2D6B', 'illinois': '#72C29F', 'california':'#64CDF0', 'fox': '#7E1939', 'texas': '#F5CF65', 'ohio': '#EF9173','florida': '#D68537'}

In [22]:
def load_and_process_topics(url_to_ts, url_to_topic_scores, n_topics=10):
    """
    Given a dict (pkl) and a mapping from url to topic scores, produce a pandas dataframe w/ columns [ts, topic1_count, topic2_count,...]

    Inputs:
        url_to_ts: dict of {url: publication ts}
        url_to_topic_scores: dict of {url: [topic1_score, topic2_score, ...]
        n_topics: number of topics; default 10

    Outputs:
        df_ts_topic: pandas dataframe w/ columns [ts, topic1_count, topic2_count,...]
    """
    ts_to_topics = {}
    for url, ts in url_to_ts.items():
        kw_scores = url_to_topic_scores[url]
        ts_round = pd.to_datetime(ts).floor('D')
        if ts_round in ts_to_topics:
            for idx, sc in enumerate(kw_scores):
                ts_to_topics[ts_round][idx] += sc
        else:
            ts_to_topics[ts_round] = {}
            for idx, sc in enumerate(kw_scores):
                ts_to_topics[ts_round][idx] = sc

    def update_ts_kw_dict(my_ts):
        """
        Given a timestamp, make a dict that contains all the keyword counts for all valid keywords

        Input: my_ts: pandas timestamp

        Output: d {keyword: count}
        """
        d = {'ts': my_ts}
        ts_topic_dict = ts_to_topics.get(my_ts, {})
        for kw in range(n_topics):
            d[topic] = ts_topics_dict.get(topic, 0)
        return d

    ts_topic_dicts = []
    for ts in list(pd.date_range(start='2023-04-01', end='2023-06-30', freq='D')):
        vals = ts_to_topics.get(ts, {})
        d = {'ts': ts}
        for i in range(n_topics):
            d[i] = vals.get(i, 0)
        ts_topic_dicts.append(d)

    return pd.DataFrame(ts_topic_dicts)


def get_media_outlet(media_group, url):
    if media_group in STATES:
        return media_group
    else:
        if 'nytimes.com' in url:
            return 'nytimes.com'
        elif 'foxnews.com' in url:
            return 'foxnews.com'
        else:
            return None



In [32]:
te_results = {i: {} for i in range(20)}
STATES = set(['newyork', 'florida', 'california', 'ohio', 'illinois', 'texas'])
N_TOPICS = 20
combined_df = pd.read_csv('/home/pranavgoel/trans-fer-entropy/topic_modeling/apr_jun_2023_post_irrelevant_article_filtering/all_texts_combined_post_irrelevant_article_filtering.csv') 
print([c for c in combined_df.columns])

print(combined_df['media_group'].unique())

combined_df['publication'] = combined_df.apply(lambda b: get_media_outlet(b.media_group, b.url), axis=1)
combined_df['media_group']=np.where(combined_df['media_name']=='New York Times','nyt',combined_df['media_group'])
combined_df['media_group']=np.where(combined_df['media_name']=='Fox News','fox',combined_df['media_group'])

['Unnamed: 0', 'media_name', 'publish_date', 'title', 'url', 'subtitle', 'text', 'sent_count', 'domain', 'gender_label', 'media_group']
['ohio' 'nytimes_foxnews' 'florida' 'texas' 'illinois' 'newyork'
 'california']


In [33]:
url_to_topic_scores = pickle.load(open('/home/pranavgoel/trans-fer-entropy/topic_modeling/apr_jun_2023_post_irrelevant_article_filtering/url_to_topic_distribution_for_num_topics_{}.pkl'.format(str(20)), 'rb'))

In [34]:
url_topic_df=pd.DataFrame(url_to_topic_scores).T.reset_index()
url_topic_df.head()

combined_df

,Unnamed: 0,media_name,publish_date,title,url,subtitle,text,sent_count,domain,gender_label,media_group,publication
0,2.0,limaohio.com,2023-04-01 15:46:07,Elida schools tackling transgender bathroom de...,https://www.limaohio.com/top-stories/2023/04/0...,NaN,ELIDA — Although parents and educators alike b...,32,limaohio.com,1,ohio,ohio
1,3.0,cleveland.com,2023-04-01 18:34:16,Only one protestor shows up for drag story hou...,https://www.cleveland.com/news/2023/04/only-on...,NaN,"CHESTERLAND, Ohio – The much-anticipated prote...",40,cleveland.com,1,ohio,ohio
2,4.0,wkbn.com,2023-04-01 18:01:43,Illinois café rescued from closure after haras...,https://www.wkbn.com/news/national-world/illin...,A suburban bakery that had planned to close it...,"LAKE IN THE HILLS, Ill. (WGN-TV) — A suburban ...",23,wkbn.com,1,ohio,ohio
3,5.0,lovelandmagazine.com,2023-04-01 11:04:53,"Ohio Right to Life makes spurious, anti-trans ...",https://www.lovelandmagazine.com/ohio-right-to...,NaN,Right to Life officials claim a reproductive r...,40,lovelandmagazine.com,1,ohio,ohio
4,6.0,cleveland.com,2023-04-02 05:00:02,Ask Amy: My conservative religious views are p...,https://www.cleveland.com/entertainment/2023/0...,NaN,Dear Amy: I am a 60-year-old conservative Cath...,50,cleveland.com,1,ohio,ohio
...,...,...,...,...,...,...,...,...,...,...,...,...
9640,7558.0,San Diego Union Tribune,2023-06-30 00:28:44.000000,Trump and DeSantis court Moms for Liberty in a...,https://www.sandiegouniontribune.com/news/nati...,The two leading contenders for the Republican ...,Former President Donald Trump poses for a phot...,49,sandiegouniontribune.com,1,california,california
9641,7560.0,http://www.hoffmania.com/,2023-06-30 14:02:00.000000,35 LGBTQ+ owned brands to support this year an...,https://www.nbcnews.com/select/shopping/lgbtq-...,We spoke to LGBTQ+ business owners about their...,Pride Month serves as a reminder of the progre...,76,nbcnews.com,1,california,california
9642,7561.0,knzr.com,2023-06-30 18:02:47.000000,Trans Influincer Lashes Out At Beer Brewer,https://www.knzr.com/trans-influincer-lashes-o...,NaN,Transgender influincer Dylan Mulvaney is addre...,9,knzr.com,1,california,california
9643,7562.0,Perris Progress,2023-06-30 12:34:48.000000,"FRIDAY June 30, 2023",https://zapinin.com/this-date-in-history/frida...,"Today in History - FRIDAY June 30, 2023 - Toda...","Today in History – FRIDAY June 30, 2023\n\nTod...",25,zapinin.com,1,california,california


In [49]:
topic_master=pd.merge(combined_df, url_topic_df,  left_on='url', right_on='index')
topic_master['publish_date']= pd.to_datetime(topic_master['publish_date'], format='mixed')

topic_master['media_group'].unique()


array(['ohio', 'texas', 'california', 'nyt', 'fox', 'florida', 'newyork',
       'illinois'], dtype=object)

# Temporal Trend

In [24]:
topic_master[list(range(0,20))].sum(axis=1)

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
9640    1.0
9641    1.0
9642    1.0
9643    1.0
9644    1.0
Length: 9645, dtype: float64

In [105]:
daily_trend_topic=topic_master.groupby(pd.Grouper(key='publish_date', freq='D'),)[list(range(0,20))].mean()
# daily_trend_topic.rename(columns=topic_labels_dict, inplace=True)

In [106]:
daily_trend_topic_long=daily_trend_topic.stack().reset_index()
daily_trend_topic_long.columns=['date','topic','mean_prob']


In [107]:
px.line(daily_trend_topic_long, x='date', y='mean_prob', color='topic').update_layout(legend=dict(
    orientation="h",
    # entrywidth=70,
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))

In [108]:
temp_area_fig=px.area(daily_trend_topic_long, x='date', y='mean_prob', color='topic', labels={''})
temp_area_fig.write_image(f'{plots_path}/temporal_area.svg')

temp_area_fig



In [109]:
line_graph=px.line(daily_trend_topic_long, x='date', y='mean_prob', color='topic', facet_col='topic', height=900, width=1200,
facet_row_spacing=0.09,
facet_col_wrap=4)

line_graph=line_graph.update_layout(showlegend=False,
).for_each_annotation(lambda a: a.update(text=topic_labels_dict[int(a.text.split("=")[-1])]))


line_graph.write_image(f'{plots_path}/temporal_line_1.svg')

In [110]:
line_graph_mono=px.line(daily_trend_topic_long, x='date', y='mean_prob', facet_col='topic', height=900, width=1200,
facet_col_wrap=4, facet_row_spacing=0.09,)

line_graph_mono=line_graph_mono.update_layout(showlegend=False,
).for_each_annotation(lambda a: a.update(text=topic_labels_dict[int(a.text.split("=")[-1])]))


line_graph_mono.write_image(f'{plots_path}/temporal_line_2.svg')

In [112]:
area_temporal_fig=px.area(daily_trend_topic_long, x='date', y='mean_prob', color='topic', facet_col='topic', facet_col_wrap=4,facet_row_spacing=0.09,
height=900, width=1200,).update_layout(showlegend=False,
).for_each_annotation(lambda a: a.update(text=topic_labels_dict[int(a.text.split("=")[-1])]))

area_temporal_fig.write_image(f'{plots_path}/temporal_area_1.svg')

area_temporal_fig

In [113]:
# px.colors.sequential

In [117]:
temporal_scatter_fig=px.scatter(daily_trend_topic_long.assign(topic=daily_trend_topic_long.topic.map(topic_labels_dict)), 
x='date', y='topic', color='mean_prob', size='mean_prob', height=1100, width=1200,
# color_continuous_scale=px.colors.sequential.Magenta,
# color_continuous_scale=px.colors.diverging.Tropic,

size_max=20,  )


temporal_scatter_fig.write_image(f'{plots_path}/temporal_scatter.svg')

temporal_scatter_fig

In [118]:
tempoeral_heatmap=px.imshow(daily_trend_topic.rename(columns=topic_labels_dict).T, height=1100,
 )


tempoeral_heatmap.write_image(f'{plots_path}/temporal_heatmap.svg')

tempoeral_heatmap

# Temporal per Collection

In [119]:
daily_collection_topic_df=topic_master.groupby([pd.Grouper(key='publish_date', freq='d'),'media_group'])[list(range(0,20))].mean().stack().reset_index(name='mean_prob')

In [120]:
topic_master['media_group'].unique()

array(['ohio', 'texas', 'california', 'nyt', 'fox', 'florida', 'newyork',
       'illinois'], dtype=object)

In [121]:
topic_master.groupby('media_group')['publish_date'].min()

media_group
california   2023-04-01 01:40:05
florida      2023-04-01 18:00:40
fox          2023-05-09 16:05:13
illinois     2023-04-01 13:06:27
newyork      2023-04-01 11:42:42
nyt          2023-04-01 09:00:07
ohio         2023-04-01 11:04:53
texas        2023-04-01 05:01:05
Name: publish_date, dtype: datetime64[ns]

In [122]:
daily_collection_topic_df.rename(columns={'level_2':'topic'}, inplace=True)
daily_collection_topic_df

,publish_date,media_group,topic,mean_prob
0,2023-04-01,california,0,0.052712
1,2023-04-01,california,1,0.000822
2,2023-04-01,california,2,0.000084
3,2023-04-01,california,3,0.053295
4,2023-04-01,california,4,0.047183
...,...,...,...,...
12895,2023-06-30,texas,15,0.086760
12896,2023-06-30,texas,16,0.022472
12897,2023-06-30,texas,17,0.298800
12898,2023-06-30,texas,18,0.038048


In [53]:
# combined_df.query('media_group == "nytimes_foxnews"')['media_name'].value_counts()

In [54]:
# combined_df.query('media_name == "Fox News"')['publish_date'].min()

In [55]:
# combined_df.query('media_name == "Fox News"')['publish_date']

In [124]:
daily_media_topic_fig=px.area(daily_collection_topic_df,
x='publish_date',
y='mean_prob',
color='media_group',
facet_row='topic',
facet_col='media_group',
height=1500,
facet_col_spacing=0.03,
facet_row_spacing=0.01,
color_discrete_map=collection_color_map,
category_orders={"media_group": [ 'nyt', 'newyork','california', 'illinois','fox','florida', 'ohio', 'texas', 
       ]},
width=1100)

daily_media_topic_fig.update_layout(showlegend=False,
# ).for_each_annotation(lambda a: a.update(text=topic_labels_dict_str.get(a.text.split("=")[-1], a.text.split("=")[-1] )))
).for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

daily_media_topic_fig.for_each_xaxis(lambda x: x.update(title=None))
daily_media_topic_fig.for_each_yaxis(lambda y: y.update(title=None))

daily_media_topic_fig.add_annotation(x=-0.05,y=0.5,
                   text="Topic Distribution", textangle=-90,
                    xref="paper", yref="paper")

daily_media_topic_fig.add_annotation(x=1.05,y=0.5,
                   text="Topic Label", textangle=-270,
                    xref="paper", yref="paper")

# daily_media_topic_fig.add_annotation(x=0.5,y=-0.05,
#                    text="Date", textangle=0,
#                     xref="paper", yref="paper")

# daily_media_topic_fig.update_layout(yaxis2=dict(title="Topic"))

daily_media_topic_fig.write_image(f'{plots_path}/temporal_collection_area.svg')
daily_media_topic_fig

# Outlet Distr

In [83]:
media_topic_df=topic_master.groupby(['media_group'])[list(range(0,20))].mean().stack().reset_index(name='mean_prob')
media_topic_df.rename(columns={'level_1':'topic'}, inplace=True)
media_topic_df

media_topic_df['topic_label']=media_topic_df['topic'].map(topic_labels_dict)
media_topic_df

,media_group,topic,mean_prob,topic_label
0,california,0,0.069291,Qender-affirming medical <br> care for transge...
1,california,1,0.021589,Pride month corporate marketing <br> backlash ...
2,california,2,0.028597,Transgender Montana house <br> representative ...
3,california,3,0.050589,Local law enforcement and <br> transgender sus...
4,california,4,0.067585,Bans on abortion and <br> gender-affirming care
...,...,...,...,...
155,texas,15,0.062889,Dylan Mulvaney Bud <br> Light backlash coverage
156,texas,16,0.029977,Housing market & policy <br> including queer b...
157,texas,17,0.037439,Supreme court rulings <br> around LGBTQ+ discr...
158,texas,18,0.048336,K-12 school policies <br> regarding LGBTQ+ stu...


In [127]:
collection_hori_bar_fig=px.bar(media_topic_df,
x='mean_prob',
y='media_group',
color='topic_label',
height=700,
text=media_topic_df['mean_prob'].map(lambda x: f"{x:.0%}"),
orientation='h')


collection_hori_bar_fig.write_image(f'{plots_path}/collection_hori_bar.svg')

collection_hori_bar_fig

In [126]:
topic_per_media=px.bar(media_topic_df,
y='mean_prob',
# x='topic_label',
color='media_group',
x='media_group',
height=900,
facet_col_wrap=4,
facet_col_spacing=0.03,
facet_row_spacing=0.08,
text=media_topic_df['mean_prob'].map(lambda x: f"{x:.0%}"),
color_discrete_map=collection_color_map,
category_orders={"media_group": [ 'nyt', 'newyork','california', 'illinois','fox','florida', 'ohio', 'texas', 
       ]},
facet_col='topic_label')

topic_per_media=topic_per_media.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))


topic_per_media.write_image(f'{plots_path}/collection_bar_facet.svg')

topic_per_media